## Assignment 4
Student: Duncan Ferguson

Student Id: 871641260

Class: Comp 4433

Assignment: Assignment 4

Date 8/9/2021

### Assignment Description
Using free and open source tools, provide a set of choropleth visualizations for each of
the columns containing dates such that the resulting visualizations (48 states only) tell
the story by conveying through color, texture, or both the time lines of achievement of
each milestone/column in the provided dataset.

Missing geoJSON_df are of particular interest in
that when a state has never achieved a given milestone, that should be indicated in a
standout manner such as cross-hatching.

Consider that the publication may be
grayscale. Provide a solution for that as well. Provide the titles, labels, and legends
necessary for clarification. File support is given as follows: SturmCodebook has the
explanation. SturmData is the geoJSON_df CSV.

#### Field Codes
state – State abbreviation

fips – FIPS State Code

icpsr – ICPSR State Code

debtfree – Year of passage of state law protecting married women’s separate property from her husband’s debts

effectivemwpa – Year of passage of state law granting married women control and management rights over their separate property

earnings – Year of passage of state law granting married women ownership of their wages or earnings on par with other separate property

wills – Year of passage of state law granting married women the ability to write wills without their husband's consent or other restrictions

soletrader – Year of passage of state law granting married women as a class the right to sign contracts and engage in business without consent of husband

Note: This study examines the passage of married women’s economic rights reforms prior to the Nineteenth Amendment (1920, granting women the right to vote at the national level), so states that passed laws later (i.e. Florida passed a control-and-management law in 1943) should be treated as missing data for the purposes of data visualization.

#### Loading our dataset

In [48]:
# importing the necessary dependencies
import pandas as pd
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import geoplot as gplt
# import geoplot.crs as gcrs
from bokeh.io import output_notebook, curdoc
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter,
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, Dropdown, Select)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import viridis
from bokeh.plotting import figure, output_file

output_notebook()

Loading BokehJS ...

In [49]:
# Importing the JSON data set
file = 'states_geo.json'
file2 = 'StrumData.csv'
file3 = 'states_id.csv'

# Importing the JSON File
with open(file) as geoJSON_df:
    dataset1 = json.load(geoJSON_df)

# Importing the Strum Dataset
with open(file2) as strum:
    df_2 = pd.read_csv(strum)

# Importing the State IDs
with open(file3) as state_id:
    df_3 = pd.read_csv(file3, usecols=[0, 1], names=["state_id", "state"], header=None, sep=',"\t')
    df_3['state'] = df_3['state'].str.strip('"')

dataset = pd.read_csv('stock_prices.csv')

# Merging Data Sets and finding Missing States
strum_w_id = df_2.merge(df_3, left_on='state', right_on='state')  # Merged Data Set
missing_states = np.setdiff1d(df_3['state'].to_list(), strum_w_id['state'].to_list()).tolist()  # Missing States

# Removing Missing states from the JSON file
datetime_cols = ['debtfree', 'effectivemwpa', 'earnings', 'wills', 'soletrader']  # columns to turn into datetime
for element in reversed(dataset['features']):
    element['state'] = df_3.iloc[element['id'], :][1]  # Add state
    add_df = strum_w_id[strum_w_id['state_id'] == element['id']]
    if element['state'] in missing_states:  # If the state is in that change list, remove it
        dataset['features'].remove(element)  # Removing states that are in the data list
    else:
        for i in add_df:
            element['properties'][i] = add_df[i].values[0]

# Reformatting the difference in the data
for element in dataset['features']:
    for i in element['properties']:
        if i in datetime_cols:
            if np.isnan(element['properties'][i]):
                element['properties'][i] = 'No Data'
            else:
                element['properties'][i] = int(element['properties'][i])

# Creating a GeoDataFrame
j_dataset = gpd.GeoDataFrame.from_features(dataset["features"])
json_data = json.dumps(json.loads(j_dataset.to_json()))
geosource = GeoJSONDataSource(geojson=json_data)

C:\Users\dunca\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


KeyError: 'features'

In [50]:
# looking at the dataset
dataset.head()
json_data.head()

NameError: name 'json_data' is not defined

Just as in the previous exercise, we want to map the date column to another column with the shortened date that only contains the year, month, and day.

In [51]:
# mapping the date of each row to only the year-month-day format
from datetime import datetime

def shorten_time_stamp(timestamp):
    shortened = timestamp[0]

    if len(shortened) > 10:
        parsed_date=datetime.strptime(shortened, '%Y-%m-%d %H:%M:%S')
        shortened=datetime.strftime(parsed_date, '%Y-%m-%d')

    return shortened

dataset['short_date'] = dataset.apply(lambda x: shorten_time_stamp(x), axis=1)

In [52]:
# looking at the dataset with shortened dat
dataset.head()
dataset1.head()

AttributeError: 'dict' object has no attribute 'head'

#### Building an interactive visualization

In [23]:
# importing the necessary dependencies
from bokeh.plotting import figure, show
from ipywidgets import interact, widgets

In [24]:
def add_candle_plot(plot, stock_name, stock_range, color):
    inc_1 = stock_range.close > stock_range.open
    dec_1 = stock_range.open > stock_range.close
    w = 0.5

    plot.segment(stock_range['short_date'], stock_range['high'],
                 stock_range['short_date'], stock_range['low'],
                 color="grey")

    plot.vbar(stock_range['short_date'][inc_1], w,
              stock_range['high'][inc_1], stock_range['close'][inc_1],
              fill_color="green", line_color="black",
              legend=('Mean price of ' + stock_name), muted_alpha=0.2)

    plot.vbar(stock_range['short_date'][dec_1], w,
              stock_range['high'][dec_1], stock_range['close'][dec_1],
              fill_color="red", line_color="black",
              legend=('Mean price of ' + stock_name), muted_alpha=0.2)

    stock_mean_val=stock_range[['high', 'low']].mean(axis=1)
    plot.line(stock_range['short_date'], stock_mean_val,
              legend=('Mean price of ' + stock_name), muted_alpha=0.2,
              line_color=color, alpha=0.5)

The last missing step is implementing the plotting of the lines if the `volume` value is selected.
We've created simple lines in the previous exercise, so this should not be a problem.

One additional interaction feature is to have an interactive legend that allows us to "mute", meaning grey out, each stock in the visualization.
To make our legend interactive please take a look at the documentation for the legend feature.
https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/legends.html

**Note:**
Don't forget to update your `add_canlde_plot` `vbar`s and `segment` to also include the `muted_alpha` parameter. Otherwise you won't be able to mute the stocks in the "open-close" visualization.

In [25]:
# method to build the plot
def get_plot(stock_1, stock_2, date, value):
    stock_1 = dataset[dataset['symbol'] == stock_1]
    stock_2 = dataset[dataset['symbol'] == stock_2]

    stock_1_name=stock_1['symbol'].unique()[0]
    stock_1_range=stock_1[(stock_1['short_date'] >= date[0]) & (stock_1['short_date'] <= date[1])]
    stock_2_name=stock_2['symbol'].unique()[0]
    stock_2_range=stock_2[(stock_2['short_date'] >= date[0]) & (stock_2['short_date'] <= date[1])]

    plot=figure(title='Stock prices',
                     x_axis_label='Date',
                     x_range=stock_1_range['short_date'],
                     y_axis_label='Price in $USD',
                     plot_width=800,
                     plot_height=500)

    plot.xaxis.major_label_orientation = 1
    plot.grid.grid_line_alpha=0.3

    if value == 'open-close':
        add_candle_plot(plot, stock_1_name, stock_1_range, 'blue')
        add_candle_plot(plot, stock_2_name, stock_2_range, 'orange')

    if value == 'volume':
        plot.line(stock_1_range['short_date'], stock_1_range['volume'],
                  legend=stock_1_name, muted_alpha=0.2)
        plot.line(stock_2_range['short_date'], stock_2_range['volume'],
                  legend=stock_2_name, muted_alpha=0.2,
                  line_color='orange')

    plot.legend.click_policy="mute"
    return plot


In [26]:
# extracing the necessary data
stock_names=dataset['symbol'].unique()
dates_2016=dataset[dataset['short_date'] >= '2016-01-01']['short_date']
unique_dates_2016=sorted(dates_2016.unique())
value_options=['open-close', 'volume']


In [47]:
# setting up the interaction elements
drp_1=widgets.Dropdown(options=stock_names,
                       value='AAPL',
                       description='Compare:')

drp_2=widgets.Dropdown(options=stock_names,
                       value='AON',
                       description='to:')

range_slider=widgets.SelectionRangeSlider(options=unique_dates_2016,
                                          index=(0,25),
                                          continuous_update=False,
                                          description='From-To',
                                          layout={'width': '500px'})

value_radio=widgets.RadioButtons(options=value_options,
                                 value='open-close',
                                 description='Metric')


In [28]:
# creating the interact method
@interact(stock_1=drp_1, stock_2=drp_2, date=range_slider, value=value_radio)
def get_stock_for_2016(stock_1, stock_2, date, value):
    show(get_plot(stock_1, stock_2, date, value))




interactive(children=(Dropdown(description='Compare:', index=4, options=('WLTW', 'A', 'AAL', 'AAP', 'AAPL', 'A…